In [1]:
import gpt_2_simple as gpt2
import os
import requests
import numpy as np
import pandas as pd
import re

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
ign_df = pd.read_pickle('ign_data/ign_data_pass_three.pkl')

ign_df = ign_df.loc[ign_df['review_score']!='null']
ign_df.reset_index(drop=True)

print(ign_df.shape)
ign_df.head()

(11072, 19)


,links,rough_review_body,cleaned_review_body,review_summary_rough,review_score,tags,categories,modified_date,published_date,author_name,tag_list,categories_list,review_title,review_body_corpus,review_score_float,number_character_review,number_words_review,polarity,subjectivity
1,https://www.ign.com/articles/2011/07/19/warham...,"\u003csection class=\""article-page\""\u003e\u00...",'Warhammer 40K: Kill Team might be unfortun...,"""And \""also-ran\"" hangs all over Warhammer 40K...",6.5,":[""review"",""blogroll"",""event-essentials"",""hot""...",":[""ign"",""ps3"",""xbox-360"",""xbox-live""],",2011-10-18T05:59:32+0000,2011-07-19T19:47:00+0000,"""Arthur Gies""","[review, blogroll, event-essentials, hot, lega...","[ign, ps3, xbox-360, xbox-live, ]","""Warhammer 40K: Kill Team Review""",warhammer 40k kill team might be unfortunat...,6.5,3705,642,0.062923,0.433371
2,https://www.ign.com/articles/2011/07/20/call-o...,"\u003csection class=\""article-page\""\u003eThe ...",The Call of Juarez franchise always intereste...,"""Call of Juarez: The Cartel is a poor change o...",4.5,":[""review"",""blogroll"",""call-of-juarez-the-cart...",":[""ign"",""pc"",""ps3"",""xbox-360""],",2011-10-18T06:02:02+0000,2011-07-20T00:14:00+0000,"""Anthony Gallegos""","[review, blogroll, call-of-juarez-the-cartel, ...","[ign, pc, ps3, xbox-360, ]","""Call of Juarez: The Cartel Review""",the call of juarez franchise always intereste...,4.5,3770,648,-0.016471,0.457898
3,https://www.ign.com/articles/2011/07/20/captai...,"\u003csection class=\""article-page\""\u003eThe ...",The majority of movie games are just awful. S...,"""Captain America: Super Soldier is a mediocre ...",5.5,":[""review"",""blogroll"",""legacy"",""wii"",""games"",""...",":[""ign"",""wii""],",2011-10-18T06:02:20+0000,2011-07-20T00:43:00+0000,"""Audrey Drake""","[review, blogroll, legacy, wii, games, captain...","[ign, wii, ]","""Captain America: Super Soldier Wii Review""",the majority of movie games are just awful st...,5.5,2675,474,-0.003842,0.544458
4,https://www.ign.com/articles/2011/07/20/quiz-c...,"\u003csection class=\""article-page\""\u003e\u00...","'Quiz Climber, from Buzz! developer 'Rele...","""Ultimately, Quiz Climber is just a little too...",6.5,":[""review"",""blogroll"",""games"",""legacy"",""wirele...",":[""ign"",""wireless""],",2011-10-18T06:07:06+0000,2011-07-20T21:44:00+0000,"""Justin Davis""","[review, blogroll, games, legacy, wireless, qu...","[ign, wireless, ]","""Quiz Climber Review""",quiz climber from buzz developer relentle...,6.5,1854,349,0.091160,0.600205
5,https://www.ign.com/articles/2011/07/20/limbo-...,"\u003csection class=\""article-page\""\u003eVide...",Video games are an art form made up of visual...,"""Limbo is an incredible achievement. Very few ...",9,":[""review"",""games"",""legacy"",""limbo"",""platforme...",":[""pc"",""ps3""],",2011-10-18T06:07:21+0000,2011-07-20T22:29:00+0000,"""Daemon Hatfield""","[review, games, legacy, limbo, platformer, pla...","[pc, ps3, ]","""Limbo Review""",video games are an art form made up of visual...,9.0,2938,527,0.096551,0.525587


In [3]:
# grab the review titles 
review_title = ign_df['review_title'].values
review_titles = [re.sub('[:,;,\\\\,\',#,\"",\/,!,?]', '', x) for x in review_title]
np.random.shuffle(review_titles)
print(review_titles)
# print(review_titles)

['My SAT Coach Review', 'The Earth Defense Force 2017 Review', 'Blood Bowl AU Review', 'Pirates Quest Hunt for Blackbeards Booty Review', 'Mystery Case Files The Malgrave Incident Review', 'Final Fantasy The 4 Heroes of Light Review', 'Crash Bandicoot Nitro Kart 2 iPhone Review', 'Marvels Guardians of the Galaxy The Telltale Series Episode 4 Who Needs You Review', 'The Suite Life of Zack and Cody Circle of Spies Review', 'Krater Shadows Over Solside Review', 'Doodle Hex UK Review', 'Green Day Rock Band Review', 'StarDrone Extreme Review', 'Ninety-Nine Nights 2 Review', 'Oblitus Review', 'Rock Band Reloaded Review', 'Elevator Action Review', 'DanceDanceRevolution Review', 'Chrono Trigger DS AU Review', 'Texas Hold em iPhone Review', 'EA Sports Active More Workouts Review', 'Deus Ex The Fall Review', 'State of Decay 2 Review', 'Game Dev Story iPhone Review', 'Sonic the Hedgehog 4 Episode I Review', 'Downhill Bowling iPhone Review', 'Mini Ninjas Review', 'Star Wars Jedi Knight II Jedi Out

In [48]:
# load model here
# training will occur in a different file
# my_run_name = 'ign_345_try01' # model name in the checkpoint

run_name = 'ign_345M_1000_epochs'
model_name = "345M"
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, 
               run_name=run_name, 
               model_name=model_name)

Loading pretrained model models\345M\model.ckpt
INFO:tensorflow:Restoring parameters from models\345M\model.ckpt


In [4]:
# loading the larger model here
run_name='774M_no_training'
model_name='774M'
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, 
               model_name=model_name)

Loading pretrained model models\774M\model.ckpt
INFO:tensorflow:Restoring parameters from models\774M\model.ckpt


In [5]:
def create_and_write_samples(prefix, length, nsamples, 
                             temperature, batch_size, 
                             write_dir, 
                             run_name, iterator):
    
    dir_path = '{}/{}'.format(write_dir, run_name)
    if not os.path.isdir(dir_path):
        os.mkdir(dir_path)
    write_path = '{}/sample_{}.txt'.format(dir_path, iterator)
    gpt2.generate(sess, 
                  run_name=run_name, 
                  model_name=model_name, 
                  prefix=prefix, 
                  temperature=temperature, 
                  length=length, 
                  nsamples=nsamples, 
                  destination_path=write_path, 
                  batch_size=batch_size) 
    
    return

In [6]:
# make function to generate several samples
i = 0

# just to randomly select what reviews to make
# review_title = ign_df['review_title'].values
# review_titles = [re.sub('"', '', x) for x in review_title]
# np.random.shuffle(review_titles)

for title in review_titles[:10]:
    create_and_write_samples(prefix=title, length=1023, 
                             nsamples=4, temperature=1.0, 
                             batch_size=1, write_dir='gpt2_generated_text', 
                             run_name=run_name, iterator=title)
    i += 1

In [40]:
raise ValueError 
# print(review_titles[:10])
# np.random.shuffle(review_titles)
test_set = review_titles[:30]
test01 = test_set[5]
print(test_set)
print(test01)
print( re.sub('[:,;,\\\\,\',#,\""]', '', test01) )

['World of Warcraft Review', 'No More Heroes Review', 'War Front: Turning Point Review', 'Diner Dash 2 Review', 'Lair Review', 'Boogie Bunnies Review', 'Baseball Blast! Review', 'Strawberry Shortcake - The Sweet Dreams Game Review', 'Lara Croft and the Guardian of Light Review', 'The Golden Compass Review', 'The Thing Review', 'Frogger Returns Review', 'Pokemon Mystery Dungeon: Explorers of Sky Review', 'NBA Ballers: Chosen One Review', 'Ghostbusters: The Video Game Review', 'CellFactor: Revolution Review', 'Castlevania: Lords of Shadow UK Review', 'Tales from the Borderlands Episode 5: The Vault of the Traveler Review', 'Pet Pals: Animal Doctor Review', 'Mini Ninjas Review', "Uncharted 3: Drake's Deception Review", 'Deal or No Deal iPhone Review', 'Siberian Strike Review', 'Ivy the Kiwi? Review', 'Outlast 2 Review', 'Art of Balance Review', 'Star Soldier R Review', 'Zen Pinball Review', 'Pictureka Review', 'Worms Review']
Boogie Bunnies Review
Boogie Bunnies Review
